In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from urllib.parse import quote
import os

In [3]:
load_dotenv(dotenv_path="password.env")

host=os.getenv("host")
port=3306
user=os.getenv("user")
database=os.getenv("name")
password=os.getenv("password")
password_embedded = quote(password)
password_embedded = quote(password)  

conn = pymysql.connect(
    host=host,
    port=3306,
    user=user,
    database=database,
    password=password)

# Connecting to SQL

connection = create_engine(f"mysql+pymysql://{user}:{password_embedded}@{host}:{port}/{database}")

In [4]:
# Reading in CSVs

inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
patrons = pd.read_csv('patrons.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')

In [5]:
# To SQL

inventory.to_sql(name="inventory", con=connection, if_exists='replace', index=False)
books.to_sql(name="books", con=connection, if_exists='replace', index=False)
authors.to_sql(name="authors", con=connection, if_exists='replace', index=False)
patrons.to_sql(name="patrons", con=connection, if_exists='replace', index=False)
loans.to_sql(name="loans", con=connection, if_exists='replace', index=False)

50

In [4]:
#Queries

query1 = "SELECT DISTINCT(Books.Title), Books.pub_year FROM Inventory JOIN Books ON Inventory.ISBN = Books.ISBN ORDER BY Books.pub_year LIMIT 5;"
query2 = "SELECT Books.Title AS Title, COUNT(Inventory.isbn) AS Count FROM Inventory LEFT JOIN Books ON Inventory.isbn = Books.isbn GROUP BY Books.Title;"
query3 = "SELECT Books.Title AS Title, COUNT(Inventory.ISBN) AS Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN LEFT JOIN Loans ON Inventory.book_id = Loans.book_id WHERE Loans.return_date IS NULL GROUP BY Books.Title;"
query4 = "SELECT Books.Genre AS Genre, COUNT(Inventory.ISBN) AS Book_Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN GROUP BY Books.Genre;"
query5 = "SELECT Books.Genre AS Genre, COUNT(DISTINCT(Books.ISBN)) AS Title_Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN GROUP BY Books.Genre;"
query6 = "SELECT Authors.language, COUNT(*) AS number FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN Authors ON Authors.author_id = Books.author_id GROUP BY Authors.language ORDER BY number;"
query7 = "SELECT `condition`, COUNT(*) FROM Inventory GROUP BY `condition`;"
query8 = "SELECT Books.title, MAX(CASE WHEN Inventory.Condition = 'As New' OR Inventory.Condition = 'good' THEN 1 ELSE 0 END) AS has_new_book FROM Books JOIN Inventory ON Books.ISBN = Inventory.ISBN GROUP BY Books.title;"
query9 = "SELECT CONCAT(Authors.first_name, ' ', Authors.last_name) AS full_name, COUNT(*) AS number_of_books FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN authors ON books.author_id = Authors.author_id GROUP BY full_name;"
query10 = "SELECT Books.Title, COUNT(*) AS loan_count FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN Loans ON Inventory.book_id = Loans.book_id GROUP BY Books.Title ORDER BY loan_count;"

In [5]:
result_df1 = pd.read_sql(query1, con=connection)
result_df1

,Title,pub_year
0,Crime and Punishment,1866
1,The Brothers Karamazov,1880
2,The Stranger,1942
3,The Plague,1947
4,Confessions of a Mask,1949


In [6]:
result_df2 = pd.read_sql(query2, con=connection)
result_df2

,Title,Count
0,East of Eden,3
1,The Stranger,2
2,Miracle Creek,1
3,FORTNITE (Official): Battle Royale Survival Guide,9
4,Meditations,2
5,1984,1
6,The Plague,2
7,The Handmaid's Tale,2
8,Crime and Punishment,2
9,Beloved,1


In [7]:
result_df3 = pd.read_sql(query3, con=connection)
result_df3

,Title,Count
0,Miracle Creek,1
1,FORTNITE (Official): Battle Royale Survival Guide,6
2,East of Eden,2
3,Meditations,1
4,Crime and Punishment,1
5,Beloved,1
6,The Alchemist,1
7,White Teeth,1
8,Half of a Yellow Sun,2
9,Americanah,2


In [8]:
result_df4 = pd.read_sql(query4, con=connection)
result_df4

,Genre,Book_Count
0,Classic,7
1,Philosophy,11
2,Thriller,2
3,Self-help,12
4,Dystopian,3
5,Historical,7
6,Literary Fiction,3
7,Science Fiction,5
8,Short Stories,3
9,Contemporary,8


In [9]:
result_df5 = pd.read_sql(query5, con=connection)
result_df5

,Genre,Title_Count
0,Classic,3
1,Contemporary,3
2,Dystopian,2
3,Historical,3
4,Historical Fiction,1
5,Literary Fiction,3
6,Magical Realism,1
7,Philosophy,5
8,Science Fiction,4
9,Self-help,3


In [10]:
result_df6 = pd.read_sql(query6, con=connection)
result_df6

,language,number
0,Korean,1
1,Greek,2
2,Spanish,2
3,Japanese,2
4,French,3
5,Portuguese,3
6,Russian,4
7,Czech,5
8,English,41


In [11]:
result_df7 = pd.read_sql(query7, con=connection)
result_df7

,condition,COUNT(*)
0,Fair,29
1,Poor,24
2,As New,2
3,Good,8


In [12]:
result_df8 = pd.read_sql(query8, con=connection)
result_df8

,title,has_new_book
0,East of Eden,0
1,The Stranger,0
2,Miracle Creek,0
3,FORTNITE (Official): Battle Royale Survival Guide,1
4,Meditations,0
5,1984,1
6,The Plague,0
7,The Handmaid's Tale,1
8,Crime and Punishment,0
9,Beloved,0


In [13]:
result_df9 = pd.read_sql(query9, con=connection)
result_df9

,full_name,number_of_books
0,John Steinbeck,3
1,George Orwell,3
2,Albert Camus,3
3,Null Epic Games,9
4,Marcus Aurelius,2
5,Margaret Atwood,3
6,Fyodor Dostoevsky,4
7,Toni Morrison,2
8,Salman Rushdie,1
9,Kazuo Ishiguro,1


In [14]:
result_df10 = pd.read_sql(query10, con=connection)
result_df10

,Title,loan_count
0,East of Eden,1
1,The House of the Spirits,1
2,Meditations,1
3,White Teeth,1
4,A Long Petal of the Sea,1
5,The Alchemist,1
6,The Plague,1
7,The Satanic Verses,1
8,Never Let Me Go,2
9,Parable of the Sower,2
